<a href="https://colab.research.google.com/github/AJoanaLlave/COM_300-Llave-Salas-Angeles-Joana/blob/main/laboratorio6_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 6 sis420 "Aprendizaje por refuerzo"
**Alumnas:** Llave Salas Ángeles Joana [link github](https://)
         Pereira Cuba Claudia [link github](https://github.com/clpereirac/SIS420-IA/tree/main/Laboratorios/Lab6AprendizajePorRefuerzo)

In [ ]:
!pip install gymnasium

PRUEBITAAAAAA

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import random

def train(episodes):
    # Se inicializa el entorno "Surround" de Atari con imagen en RGB
    env = gym.make("ALE/Surround-v5", obs_type="rgb")

    # Obtenemos las dimensiones del espacio de observación y el tamaño del espacio de acción
    state_shape = env.observation_space.shape
    action_size = env.action_space.n

    # Creamos la tabla Q inicializada en cero. Las filas son los estados y las columnas son las acciones
    q_table = np.zeros((210, 160, action_size))

    # Parámetros de Q-learning
    learning_rate = 0.3  # Tasa de aprendizaje
    discount_factor = 0.9  # Factor de descuento (para recompensas futuras)
    epsilon = 1.0  # Probabilidad de explorar (acciones aleatorias)
    epsilon_decay_rate = 0.0003  # Decaimiento de epsilon, para que explore menos conforme pasa el tiempo
    rng = np.random.default_rng()  # Generador de números aleatorios

    # Array para guardar las recompensas de cada episodio
    rewards_por_episode = np.zeros(episodes)

    # Bucle de entrenamiento (todos los episodios)
    for i in range(episodes):
        # Reiniciamos el entorno
        env.close()
        if (i + 1) % 100 == 0:
            # Cada 100 episodios, mostramos la simulación (rendering)
            env = gym.make("ALE/Surround-v5", obs_type="rgb", render_mode="human")
        else:
            env = gym.make("ALE/Surround-v5", obs_type="rgb")

        # Obtenemos el estado inicial y lo convertimos a escala de grises (simplificación)
        state, _ = env.reset()
        state = state.mean(axis=2).astype(int)  # Promediamos los 3 canales de color (RGB) para simplificar

        # Variables para saber si el episodio terminó o fue truncado
        terminated = False
        truncated = False

        # Bucle de cada paso en el episodio
        while not terminated and not truncated:
            # Si epsilon es alto, exploramos (seleccionamos una acción aleatoria), si no, explotamos (seleccionamos la mejor acción según la tabla Q)
            if rng.random() < epsilon:
                action = env.action_space.sample()  # Exploración (acción aleatoria)
            else:
                # Explotación: seleccionamos la acción que tiene el mejor valor de Q en la tabla
                action = np.argmax(q_table[state.mean(axis=0), state.mean(axis=1)])

            # Realizamos la acción y obtenemos el nuevo estado y la recompensa
            new_state, reward, terminated, truncated, _ = env.step(action)
            new_state = new_state.mean(axis=2).astype(int)  # Convertimos a escala de grises

            # Actualizamos la tabla Q usando la fórmula estándar de Q-learning
            best_next_action = np.argmax(q_table[new_state.mean(axis=0), new_state.mean(axis=1)])
            q_table[state.mean(axis=0), state.mean(axis=1), action] += learning_rate * (
                reward + discount_factor * q_table[new_state.mean(axis=0), new_state.mean(axis=1), best_next_action]
                - q_table[state.mean(axis=0), state.mean(axis=1), action]
            )

            # El nuevo estado pasa a ser el actual
            state = new_state

        # Reducimos epsilon poco a poco para hacer que el agente explore menos conforme aprende
        epsilon = max(epsilon - epsilon_decay_rate, 0.01)

        # Guardamos la recompensa obtenida en este episodio
        rewards_por_episode[i] = reward

        # Imprimimos un poco el progreso cada 50 episodios
        if (i + 1) % 50 == 0:
            print(f"Episodio: {i + 1} - Recompensa: {rewards_por_episode[i]}")

    # Cerramos el entorno después de entrenar
    env.close()

    # Mostramos la tabla Q final después de todo el entrenamiento
    print("Tabla Q resultante después del entrenamiento:")
    print(q_table)

    # Calculamos y graficamos la suma de las recompensas acumuladas por cada bloque de 50 episodios
    suma_rewards = np.zeros(episodes)
    for t in range(episodes):
        suma_rewards[t] = np.sum(rewards_por_episode[max(0, t - 50):(t + 1)])

    # Graficamos las recompensas acumuladas a lo largo de los episodios
    plt.plot(suma_rewards)
    plt.xlabel('Episodios')
    plt.ylabel('Suma de recompensas acumuladas')
    plt.title('Evolución de las recompensas acumuladas durante el entrenamiento')
    plt.show()

if __name__ == "__main__":
    # Entrenamos el modelo con 5000 episodios
    train(5000)


NamespaceNotFound: Namespace ALE not found. Have you installed the proper package for ALE?